In [3]:
# Lib imports
import os
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras import layers, models
from tensorflow.keras.applications import ResNet50
import numpy as np

In [4]:
#DATASET DIRECTORY CONFIGURATION
#Download and unzip the dataset from Kagglee, set the directory paths accordingly

train_dir = "animals"
test_dir = "animals"

In [5]:
# Image Parameters
# Used to resize the input images, also will determine the input size of layer.

IMG_SIZE = (224, 224)
BATCH_SIZE = 32

In [6]:
train_datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.15,
    zoom_range=0.2,
    horizontal_flip=True,
    validation_split=0.2
)
test_datagen = ImageDataGenerator(rescale=1./255)

train_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size=IMG_SIZE,
    batch_size=BATCH_SIZE,
    class_mode='binary',
    subset='training'
)
val_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size=IMG_SIZE,
    batch_size=BATCH_SIZE,
    class_mode='binary',
    subset='validation'
)
test_generator = test_datagen.flow_from_directory(
    test_dir,
    target_size=IMG_SIZE,
    batch_size=BATCH_SIZE,
    class_mode='binary',
    shuffle=False
)

Found 800 images belonging to 2 classes.
Found 199 images belonging to 2 classes.
Found 999 images belonging to 2 classes.


In [7]:
# ResNet 50 with transfer learning

base_model = ResNet50(
    input_shape =(IMG_SIZE[0], IMG_SIZE[1], 3),
    include_top = False,
    weights = 'imagenet'
)

# Freeze the base model layers
base_model.trainable = False

# Build the model
model = models.Sequential([
    base_model,
    layers.GlobalAveragePooling2D(),
    layers.Dense(256, activation='relu'),
    layers.Dropout(0.5),
    layers.Dense(1, activation='sigmoid')
])

# Compile the model
optimizer = tf.keras.optimizers.Adam(learning_rate=0.001)
model.compile(optimizer = optimizer, loss = 'binary_crossentropy', metrics = ['accuracy'])

# Display the model
model.summary()

Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ resnet50 (Functional)           │ (None, 7, 7, 2048)     │    23,587,712 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ global_average_pooling2d        │ (None, 2048)           │             0 │
│ (GlobalAveragePooling2D)        │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 256)            │       524,544 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout (Dropout)               │ (None, 256)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 1)              │           257 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 24,112,513 (91.98 MB)

 Trainable params: 524,801 (2.00 MB)

 Non-trainable params: 23,587,712 (89.98 MB)

In [8]:
# Train the model
history = model.fit(
    train_generator,
    epochs = 15,
    validation_data = val_generator
)

Epoch 1/15
25/25 ━━━━━━━━━━━━━━━━━━━━ 36s 1s/step - accuracy: 0.5113 - loss: 0.8522 - val_accuracy: 0.5980 - val_loss: 0.6759
Epoch 2/15
25/25 ━━━━━━━━━━━━━━━━━━━━ 34s 1s/step - accuracy: 0.5263 - loss: 0.7090 - val_accuracy: 0.5980 - val_loss: 0.6686
Epoch 3/15
25/25 ━━━━━━━━━━━━━━━━━━━━ 34s 1s/step - accuracy: 0.5562 - loss: 0.6984 - val_accuracy: 0.5980 - val_loss: 0.6737
Epoch 4/15
25/25 ━━━━━━━━━━━━━━━━━━━━ 34s 1s/step - accuracy: 0.6225 - loss: 0.6620 - val_accuracy: 0.5779 - val_loss: 0.6608
Epoch 5/15
25/25 ━━━━━━━━━━━━━━━━━━━━ 34s 1s/step - accuracy: 0.5838 - loss: 0.6706 - val_accuracy: 0.6683 - val_loss: 0.6525
Epoch 6/15
25/25 ━━━━━━━━━━━━━━━━━━━━ 34s 1s/step - accuracy: 0.6175 - loss: 0.6574 - val_accuracy: 0.6884 - val_loss: 0.6383
Epoch 7/15
25/25 ━━━━━━━━━━━━━━━━━━━━ 33s 1s/step - accuracy: 0.6175 - loss: 0.6617 - val_accuracy: 0.6332 - val_loss: 0.6511
Epoch 8/15
25/25 ━━━━━━━━━━━━━━━━━━━━ 34s 1s/step - accuracy: 0.6438 - loss: 0.6416 - val_accuracy: 0.6633 - val_loss:

In [9]:
# Unfreeze the last few layers of ResNet50 for fine tuning
base_model.trainable = True

#Freeze all layers except the last 20
for layer in base_model.layers[:-50]:
    layer.trainable = False

# Recompile with lower learning rate for fine-tuning
optimizer = tf.keras.optimizers.Adam(learning_rate = 0.0001)
model.compile(optimizer = optimizer, loss = 'binary_crossentropy', metrics = ['accuracy'])

# Continue training with fine-tuning
history_fine = model.fit(
    train_generator,
    epochs = 15,
    validation_data = val_generator
)

Epoch 1/15
25/25 ━━━━━━━━━━━━━━━━━━━━ 59s 2s/step - accuracy: 0.7387 - loss: 0.5374 - val_accuracy: 0.5025 - val_loss: 0.7173
Epoch 2/15
25/25 ━━━━━━━━━━━━━━━━━━━━ 46s 2s/step - accuracy: 0.8100 - loss: 0.4288 - val_accuracy: 0.6784 - val_loss: 0.6217
Epoch 3/15
25/25 ━━━━━━━━━━━━━━━━━━━━ 47s 2s/step - accuracy: 0.8300 - loss: 0.3986 - val_accuracy: 0.5930 - val_loss: 0.6442
Epoch 4/15
25/25 ━━━━━━━━━━━━━━━━━━━━ 255s 11s/step - accuracy: 0.8550 - loss: 0.3617 - val_accuracy: 0.5126 - val_loss: 1.5408
Epoch 5/15
25/25 ━━━━━━━━━━━━━━━━━━━━ 277s 7s/step - accuracy: 0.8462 - loss: 0.3661 - val_accuracy: 0.6985 - val_loss: 0.5483
Epoch 6/15
25/25 ━━━━━━━━━━━━━━━━━━━━ 46s 2s/step - accuracy: 0.8687 - loss: 0.2999 - val_accuracy: 0.7035 - val_loss: 0.7385
Epoch 7/15
25/25 ━━━━━━━━━━━━━━━━━━━━ 48s 2s/step - accuracy: 0.8550 - loss: 0.3274 - val_accuracy: 0.8392 - val_loss: 0.3674
Epoch 8/15
25/25 ━━━━━━━━━━━━━━━━━━━━ 48s 2s/step - accuracy: 0.8737 - loss: 0.3098 - val_accuracy: 0.6683 - val_lo

In [10]:
# Evaluate the model

test_loss, test_acc = model.evaluate(test_generator)
print(f"Test Accuracy: {test_acc}")

32/32 ━━━━━━━━━━━━━━━━━━━━ 101s 3s/step - accuracy: 0.9329 - loss: 0.1640
Test Accuracy: 0.9329329133033752


In [11]:
# SAVE THE MODEL
model.save('exercise_6_resnet_brey.h5')

In [12]:
# SIMPLE INFERENCE SCRIPT
from tensorflow.keras.preprocessing import image

def predict_image(img_path, model_path='exercise_6_resnet_brey.h5'):
    model = tf.keras.models.load_model(model_path)
    img = image.load_img(img_path, target_size=IMG_SIZE)
    img_array = image.img_to_array(img) / 255.0
    img_array = np.expand_dims(img_array, axis=0)
    pred = model.predict(img_array)[0,0]
    label = "Dog" if pred >= 0.5 else "Cat"
    confidence = pred if pred >= 0.5 else 1 - pred
    print(f"Prediction: {label} (confidence: {confidence:.2%})")
    return label, confidence

In [13]:
# Example usage:
predict_image("run/run_1a.png")
predict_image("run/run_1b.png")

1/1 ━━━━━━━━━━━━━━━━━━━━ 5s 5s/step
Prediction: Dog (confidence: 98.66%)


1/1 ━━━━━━━━━━━━━━━━━━━━ 5s 5s/step
Prediction: Cat (confidence: 100.00%)


('Cat', np.float32(0.9999988))